In [16]:
import numpy as np
import os 
import cv2
import random
import sys 
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import bz2
from collections import defaultdict
from bz2 import BZ2File
from datetime import datetime
from PIL import Image
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.metrics import auc, confusion_matrix
import sys
from scikit_posthocs import posthoc_ttest
sys.path.append('.')
from src.modules import *

In [17]:
def statistics(results_all, bs_num=50):
    
    np.random.seed(10)
    
    perf_models = {}
    for j in range(len(results_all['attr0'])):
        perf_models[f'perf_bs{j}'] = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
    
    for k in range(bs_num):
        
#         num_sample = len(results_all[f'attr{i}'][j][0])
#         class0idx = np.where(results_all['attr3'][10][0]==0)[0]
#         class1idx = np.where(results_all['attr3'][10][0]==1)[0]
#         idx0 = np.random.choice(class0idx, len(class0idx))
#         idx1 = np.random.choice(class1idx, len(class1idx))
#         idx = list(idx0) + list(idx1)
        
        for i in range(7):
            
            class0idx = np.where(results_all[f'attr{i}'][10][0]==0)[0]
            class1idx = np.where(results_all[f'attr{i}'][10][0]==1)[0]
            idx0 = np.random.choice(class0idx, len(class0idx))
            idx1 = np.random.choice(class1idx, len(class1idx))
            idx = list(idx0) + list(idx1)
            
            for j in range(len(perf_models)):
                results_epoch = results_all[f'attr{i}'][j]
                
                if len(results_epoch[0]) < 3000:
                    class0idx = np.where(results_all[f'attr{i}'][j][0]==0)[0]
                    class1idx = np.where(results_all[f'attr{i}'][j][0]==1)[0]
                    idx0 = np.random.choice(class0idx, len(class0idx))
                    idx1 = np.random.choice(class1idx, len(class1idx))
                    idx = list(idx0) + list(idx1)
                
                _gt = results_epoch[0][idx]
                _pred = results_epoch[1][idx]
                _attr = results_epoch[2][:, idx]

                val_es_acc, val_es_auc, val_aucs_by_attrs, val_dpds, val_eods, val_between_group_disparity = evalute_comprehensive_perf(_pred,
                                                                                                                                    _gt,
                                                                                                                                    _attr)

                perfi = [val_es_acc[i], val_es_auc[i]]
                perfi.extend(val_aucs_by_attrs[i])
                perfi.append(val_dpds[i])
                perfi.append(val_eods[i])
                perfi.extend(val_between_group_disparity[i])
#                 perf_bs_proposed[f'attr{i}'].append(perfi)

                perf_models[f'perf_bs{j}'][f'attr{i}'].append(perfi)
    
    attr_pvalues = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
    
    for i in range(7):
        num_metrics = len(perf_models['perf_bs0'][f'attr{i}'][0])
        for k in range(num_metrics):
            vpaired = []
            for j in range(len(perf_models.keys())):
                vpaired.append(np.array(perf_models[f'perf_bs{j}'][f'attr{i}'])[:, k])
            
            vpaired = np.array(vpaired)
            vpaired = vpaired[:, ~np.isnan(vpaired).any(axis=0)]
            
            pvalues = posthoc_ttest(vpaired)
            
            attr_pvalues[f'attr{i}'].append(pvalues)
            
    return attr_pvalues

In [18]:
# # transform adversarial results

folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_adversarial'

attrs = ['race', 'gender', 'ethnicity', 'maritalstatus', 'language', 'gender_race', 'gender_ethnicity']
attrmaps = {'attr0':'Race', 'attr1':'Gender', 'attr2':'Ethnicity', 'attr3':'Maritalstatus', 'attr4':'Language', 
                'attr5':'Gender+Race', 'attr6':'Gender+Ethnicity'}


outcome_adv = {'vgg':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'efficientnet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'resnet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'swin':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'convnext':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'densenet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'ViT-B':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}}

for idx, attr in enumerate(attrs):
    modality = f'dr_slo_fundus_{attr}'

    modality_folder = f'{folder}/{modality}'

    bestresults = []
    epochresults = []
    for f in os.listdir(modality_folder):
        if f.endswith('.csv'):
            paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
            bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
        else:
            epochresults.append(f)
    pairedresults = {}
    for i, v in enumerate(bestresults):
        for p in epochresults:
            if v[1] in p:
                pairedresults[i] = [v[0], p]
    
    for k in pairedresults.keys():
        v = pairedresults[k]
    
        resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
        results = {}
        attrgroupnums = {}
        for name, data in resultpd.items():
            results[name.strip()] = data.values[-1]
            if 'group' in name:
                attri = name.split('_')[1]
                if attri not in attrgroupnums.keys():
                    attrgroupnums[attri] = 1
                else:
                    attrgroupnums[attri] += 1
        modelname = v[0].split('_')[1]
        
        
        vs = [results[f'esacc_attr{idx}'], results['acc'], results[f'esauc_attr{idx}'], results['auc']]
        vs = vs + [results[f'auc_attr{idx}_group{j}'] for j in range(attrgroupnums[f'attr{idx}'])]
        vs = vs + [results[f'dpd_attr{idx}'], results[f'eod_attr{idx}'], 
                   results[f'std_group_disparity_attr{idx}'],
                   results[f'max_group_disparity_attr{idx}']]
        
        npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
        test_gt = npzdata['test_gt']
        test_pred = npzdata['test_pred']
        test_attr = npzdata['test_attr']
        vs_epoch = [test_gt, test_pred, test_attr]
        
#         print(len(test_gt), len(test_pred), len(test_attr), os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
       
        outcome_adv[modelname][f'attr{idx}'] = (vs, vs_epoch)
    

In [20]:
# # transform proposed model results

folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis'

attrs = ['race', 'gender', 'ethnicity', 'maritalstatus', 'language', 'genderrace', 'genderethnicity']
attrmaps = {'attr0':'Race', 'attr1':'Gender', 'attr2':'Ethnicity', 'attr3':'Maritalstatus', 'attr4':'Language', 
                'attr5':'Gender+Race', 'attr6':'Gender+Ethnicity'}


outcome_fis = {'efficientnet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'ViT-B':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}}

for idx, attr in enumerate(attrs):
    modality = f'dr_slo_fundus_{attr}_fis'

    modality_folder = f'{folder}/{modality}'

    print(modality_folder)
    
    bestresults = []
    epochresults = []
    for f in os.listdir(modality_folder):
        if f.endswith('.csv'):
            paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
            bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
        else:
            epochresults.append(f)
    pairedresults = {}
    for i, v in enumerate(bestresults):
        for p in epochresults:
            if v[1] in p:
                pairedresults[i] = [v[0], p]
    
    for k in pairedresults.keys():
        v = pairedresults[k]
    
        resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
        results = {}
        attrgroupnums = {}
        for name, data in resultpd.items():
            results[name.strip()] = data.values[-1]
            if 'group' in name:
                attri = name.split('_')[1]
                if attri not in attrgroupnums.keys():
                    attrgroupnums[attri] = 1
                else:
                    attrgroupnums[attri] += 1
        modelname = v[0].split('_')[1]
        
        
        vs = [results[f'esacc_attr{idx}'], results['acc'], results[f'esauc_attr{idx}'], results['auc']]
        vs = vs + [results[f'auc_attr{idx}_group{j}'] for j in range(attrgroupnums[f'attr{idx}'])]
        vs = vs + [results[f'dpd_attr{idx}'], results[f'eod_attr{idx}'], 
                   results[f'std_group_disparity_attr{idx}'],
                   results[f'max_group_disparity_attr{idx}']]
        
        npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
        test_gt = npzdata['test_gt']
        test_pred = npzdata['test_pred']
        test_attr = npzdata['test_attr']
        vs_epoch = [test_gt, test_pred, test_attr]
        
#         print(len(test_gt), len(test_pred), len(test_attr), os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
       
        outcome_fis[modelname][f'attr{idx}'] = (vs, vs_epoch)
    

/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_race_fis
/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_gender_fis
/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_ethnicity_fis
/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_maritalstatus_fis
/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_language_fis
/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_genderrace_fis
/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_genderethnicity_fis


In [21]:
os.listdir('/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL/dr_slo_fundus_fis/dr_slo_fundus_ethnicity_fis')

['.ipynb_checkpoints',
 'best_ViT-B_slo_fundus_hispanic.csv',
 'best_efficientnet_slo_fundus_hispanic.csv',
 'efficientnet_slo_fundus_lr1e-4_bz10_seed9036_auc0.8112',
 'ViT-B_slo_fundus_lr1e-4_bz64_seed13_auc0.8616']

In [15]:
# statistic analysis
modality = 'slo_fundus' #'slo_fundus or oct_bscans'
folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_harvard10k_FINAL'


modality_folder = f'{folder}/dr_{modality}_race'

model_order = ['vgg', 'swin', 'resnet', 'convnext', 'densenet', 'efficientnet', 'ViT-B', 
               'vgg_oversample','swin_oversample', 'resnet_oversample', 'convnext_oversample', 
               'densenet_oversample', 'efficientnet_oversample', 'ViT-B_oversample']

attrmaps = {'attr0':'Race', 'attr1':'Gender', 'attr2':'Ethnicity', 'attr3':'Maritalstatus', 'attr4':'Language', 
            'attr5':'Gender+Race', 'attr6':'Gender+Ethnicity'}

bestresults = []
epochresults = []
for f in os.listdir(modality_folder):
    if f.endswith('.csv'):
        print(f)
        paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
        bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
    else:
        epochresults.append(f)
        
pairedresults = {}
modelindex = {}
for i, v in enumerate(bestresults):
    for p in epochresults:
        if v[1] in p:
            pairedresults[i] = [v[0], p]
            modelname = v[0].split('_')[1]
            if 'oversample' in v[0]:
                modelname = modelname + '_oversample'
            modelindex[modelname] = i

best_results_all = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
column_names = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
rownames = []
pred_results_all = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}

num_attrs = 7

for k in model_order:
    if k not in modelindex.keys():
        continue
    k = modelindex[k]
    v = pairedresults[k]
    resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
    results = {}
    attrgroupnums = {}
    for name, data in resultpd.items():
        results[name.strip()] = data.values[-1]
        if 'group' in name:
            attri = name.split('_')[1]
            if attri not in attrgroupnums.keys():
                attrgroupnums[attri] = 1
            else:
                attrgroupnums[attri] += 1
    modelname = v[0].split('_')[1]
    if 'oversample' in v[0]:
        modelname = modelname + '_oversample'
    
    rownames.append(modelname)
    print(modelname)
    npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
    test_gt = npzdata['test_gt']
    test_pred = npzdata['test_pred']
    test_attr = npzdata['test_attr']
    
    for i in range(num_attrs):
        vs = [results[f'esacc_attr{i}'], results['acc'], results[f'esauc_attr{i}'], results['auc']]
        vs = vs + [results[f'auc_attr{i}_group{j}'] for j in range(attrgroupnums[f'attr{i}'])]
        vs = vs + [results[f'dpd_attr{i}'], results[f'eod_attr{i}'], 
                   results[f'std_group_disparity_attr{i}'],
                   results[f'max_group_disparity_attr{i}']]
        best_results_all[f'attr{i}'].append(vs)
        
        vsname = ['esacc', 'acc', 'esauc', 'auc']
        vsname = vsname + [f'auc_group{j}' for j in range(attrgroupnums[f'attr{i}'])]
        vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
        column_names[f'attr{i}'] = vsname

        pred_results_all[f'attr{i}'].append([test_gt, test_pred, test_attr])


# append adversarial outcome
modeladv_order = ['vgg', 'swin', 'resnet', 'convnext', 'densenet', 'efficientnet', 'ViT-B']
for modelname in modeladv_order:
    modelname_subfix = f'{modelname}_adv'
    rownames.append(modelname_subfix)
    print(modelname_subfix)
    for attri in outcome_adv[modelname].keys():
#         try:
        best_results_all[attri].append(outcome_adv[modelname][attri][0])
        pred_results_all[attri].append(outcome_adv[modelname][attri][1])
#         except: 
#             pass


# append proposed model outcome
modelfis_order = ['efficientnet', 'ViT-B']
for modelname in modelfis_order:
    modelname_subfix = f'{modelname}_fis'
    rownames.append(modelname_subfix)
    print(modelname_subfix)
    for attri in outcome_fis[modelname].keys():
#         try:
        best_results_all[attri].append(outcome_fis[modelname][attri][0])
        pred_results_all[attri].append(outcome_fis[modelname][attri][1])
#         except: 
#             pass
    
    
# write best outcome    
writer = pd.ExcelWriter(f"{folder}/dr_{modality}_race_combined_outcome_FINAL.xlsx", engine="xlsxwriter")

for k in attrmaps.keys():
    attridx = int(k[-1])
    pdattri = pd.DataFrame(best_results_all[f'attr{attridx}'], 
                 index=rownames, 
                 columns=column_names[f'attr{attridx}'])
    pdattri.to_excel(writer, sheet_name=attrmaps[k])

    
attr_pvalues = statistics(pred_results_all)

for i in range(num_attrs):
    rowidx = 0
    vsname = ['esacc', 'acc', 'esauc', 'auc']
    vsname = vsname + [f'auc_group{j}' for j in range(len(attr_pvalues[f'attr{i}']))]
    vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
    
    for idx, pvalues in enumerate(attr_pvalues[f'attr{i}']):
        
        rownames_prefix = [str(v)+'@'+vsname[idx] for v in rownames]
        
        pvdf = pd.DataFrame(np.array(pvalues), index=rownames_prefix, columns=rownames_prefix)
        
        pvdf.to_excel(writer, sheet_name=attrmaps[f'attr{i}']+'_statistics', startrow=rowidx)
        rowidx += len(pvalues) + 2
    
writer.close()
# pairedresults

best_ViT-B_slo_fundus_race.csv
best_swin_slo_fundus_race.csv
best_resnet_slo_fundus_race_oversample.csv
best_ViT-B_slo_fundus_race_oversample.csv
best_convnext_slo_fundus_race.csv
best_vgg_slo_fundus_race.csv
best_swin_slo_fundus_race_oversample.csv
best_efficientnet_slo_fundus_race.csv
best_efficientnet_slo_fundus_race_oversample.csv
best_vgg_slo_fundus_race_oversample.csv
best_densenet_slo_fundus_race_oversample.csv
best_resnet_slo_fundus_race.csv
best_convnext_slo_fundus_race_oversample.csv
best_densenet_slo_fundus_race.csv
vgg
swin
resnet
convnext
densenet
efficientnet
ViT-B
vgg_oversample
swin_oversample
resnet_oversample
convnext_oversample
densenet_oversample
efficientnet_oversample
ViT-B_oversample
vgg_adv
swin_adv
resnet_adv
convnext_adv
densenet_adv
efficientnet_adv
ViT-B_adv
efficientnet_fis
ViT-B_fis


/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

...

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

In [14]:
attridx = 0
pd.DataFrame(best_results_all[f'attr{attridx}'], index=rownames, columns=column_names[f'attr{attridx}'])

,esacc,acc,esauc,auc,auc_group0,auc_group1,auc_group2,dpd,eod,std_group_disparity,max_group_disparity
vgg,0.7796,0.8520,0.6507,0.7168,0.6336,0.7337,0.7153,0.0691,0.1648,0.0607,0.1396
swin,0.8722,0.9247,0.6572,0.8170,0.6457,0.7655,0.8375,0.0187,0.2224,0.0968,0.2347
resnet,0.8549,0.9137,0.6525,0.7505,0.6411,0.7214,0.7622,0.0009,0.0108,0.0670,0.1614
convnext,0.7806,0.8910,0.6317,0.7915,0.6221,0.7302,0.8137,0.0474,0.2157,0.0991,0.2420
densenet,0.8444,0.9120,0.6427,0.7769,0.6094,0.7487,0.7901,0.0255,0.1472,0.0995,0.2325
efficientnet,0.8716,0.9217,0.6713,0.7939,0.7272,0.7002,0.8162,0.0020,0.1687,0.0624,0.1461
ViT-B,0.8486,0.9223,0.7652,0.8570,0.8065,0.8004,0.8698,0.0099,0.2708,0.0366,0.0810
vgg_oversample,0.6566,0.7473,0.6345,0.7082,0.6002,0.7114,0.7131,0.0956,0.1218,0.0746,0.1593
swin_oversample,0.8569,0.9190,0.6287,0.7810,0.6501,0.6982,0.8095,0.0118,0.2473,0.0855,0.2042
resnet_oversample,0.8545,0.9130,0.6659,0.7305,0.6639,0.7085,0.7390,0.0000,0.0000,0.0422,0.1028
